In [ ]:
#This is the file I used to run blobs of segments files

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sc
from scipy.signal import find_peaks, peak_widths
from scipy.stats import skew, kurtosis
from scipy.integrate import simps
from sklearn.preprocessing import MinMaxScaler

import os

In [2]:
def add_gender_age(df,filename):
    if 'P5' in filename:
        df['gender'] = 'M'
        df['age'] = 10
    if 'P6' in filename:
        df['gender'] = 'M'
        df['age'] = 8
    if 'P7' in filename:
        df['gender'] = 'M'
        df['age'] = 7
    if 'P8' in filename:
        df['gender'] = 'F'
        df['age'] = 6
        
    if 'P9' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    if 'P10' in filename:
        df['gender'] = 'F'
        df['age'] = 10
    if 'P11' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    if 'P12' in filename:
        df['gender'] = 'M'
        df['age'] = 11
        
    if 'P13' in filename:
        df['gender'] = 'M'
        df['age'] = 8
        
    if 'P14' in filename:
        df['gender'] = 'M'
        df['age'] = 10
    if 'P15' in filename:
        df['gender'] = 'M'
        df['age'] = 8
        
    if 'P16' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    if 'P17' in filename:
        df['gender'] = 'M'
        df['age'] = 9
    if 'P18' in filename:
        df['gender'] = 'M'
        df['age'] = 6
        
    if 'P19' in filename:
        df['gender'] = 'M'
        df['age'] = 11
    if 'P20' in filename:
        df['gender'] = 'F'
        df['age'] = 11
    if 'P21' in filename:
        df['gender'] = 'F'
        df['age'] = 10
        
    if 'P22' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    if 'P23' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    if 'P24' in filename:
        df['gender'] = 'F'
        df['age'] = 7
    return df

def add_specific_hyperactive_behavior(df,filename):
    file_components = filename.split('_')
    if 'NoHyp' in file_components:
        df['specific_hyp'] = 0
    if 'peers' in file_components:
        df['specific_hyp'] = 1
    if 'self' in file_components:
        df['specific_hyp'] = 2
    if 'Twirling' in file_components:
        df['specific_hyp'] = 3
    if 'chair' in file_components:
        df['specific_hyp'] = 4
    if 'Manipulating' in file_components:
        df['specific_hyp'] = 5
    if 'Standing' in file_components:
        df['specific_hyp'] = 6
    if 'Drumming_fingers' in filename:
        df['specific_hyp'] = 7
    if 'Bouncing_your_leg' in filename:
        df['specific_hyp'] = 8
    if 'Finger_tapping' in filename:
        df['specific_hyp'] = 9
    if 'Tapping_foot' in filename:
        df['specific_hyp'] = 10
    if 'Shaking_leg' in filename:
        df['specific_hyp'] = 11
    return df

In [3]:
def moving_average(X, window_size):
    X_new =[]
    # end_index = window_size
    for i, value in enumerate(X):
        #print("index, value",i, value)
        X_new.insert(i, sum(X[i:i+window_size])/len(X[i:i+window_size]))
    return X_new

def moving_average_dataframe(X, window_size):
    X_new = pd.DataFrame()  # Create an empty DataFrame to store the results
    for column in X.columns:
        X_new[column] = X[column].rolling(window=window_size).mean()
    X_new = X_new.dropna()
    return X_new

In [4]:
def calculate_sampling_frequency(dataframe):
    frequencies = dataframe['timestamp_str_seconds'].value_counts()
    average_frequency = sum(frequencies) / len(frequencies)

    return average_frequency


def calculate_fft(data, sampling_rate):
    data_no_dc = data - data.mean()
    fft_result = np.fft.fft(data_no_dc, axis=0)
    #freqs = np.fft.fftfreq(len(fft_result), 1/sampling_rate)
    return pd.DataFrame(abs(fft_result),  columns=data.columns)

def dominant_frequency(data, sampling_rate, feature_name):
    data_no_dc = data - data.mean()
    fft_result = np.fft.fft(data_no_dc, axis=0)
    freqs = np.fft.fftfreq(len(fft_result), 1/sampling_rate)
    dominant_freq = np.abs(freqs[np.argmax(np.abs(fft_result), axis=0)])
    dominant_df = pd.DataFrame(dominant_freq).transpose()
    dominant_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name, "Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name]
    return dominant_df


def maximum_amplitude(data, sampling_rate, feature_name):
    data_no_dc = data - data.mean()
    fft_result = np.fft.fft(data_no_dc, axis=0)
    #freqs = np.fft.fftfreq(len(fft_result), 1/sampling_rate)
    # Calculate maximum amplitude for each column
    max_amplitude = np.max(np.abs(fft_result), axis=0)
    amplitude_df = pd.DataFrame(max_amplitude).transpose()
    amplitude_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name, "Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name]
    return amplitude_df


def calculate_q_factor(data, sampling_rate, feature_name):
    # Find peaks in the data
    q_factors = []

    for column_name in data.columns:
        column_data = data[column_name].values  # Extract column as a NumPy array

        # Find peaks in the data
        peaks, _ = find_peaks(column_data)

        if len(peaks) < 1:
            q_factors.append(None)  # No peaks found for this column
            continue

        # Calculate peak width
        widths, _, _, _ = peak_widths(column_data, peaks)

        if len(widths) < 1:
            q_factors.append(None)  # Unable to calculate Q factor for this column
            continue

        # Calculate Q factor using the formula: f_peak / width
        max_index = np.argmax(column_data)
        peak_frequency = peaks[np.argmin(np.abs(peaks - max_index))]  # Find the nearest peak
        q_factor = peak_frequency / widths[0]

        # Append the calculated Q factor to the list
        q_factors.append(q_factor)
        
    q_factor_df = pd.DataFrame(q_factors).transpose()
    q_factor_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name, "Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name]
    return q_factor_df


def calculate_auc_feature(feature_name, accel_gyro_data):
    freq = accel_gyro_data.index
    
    # Use NumPy vectorized operations to calculate AUC for each column
    auc_values = np.trapz(accel_gyro_data.values, freq, axis=0)
    
    columns = [f"{col}_{feature_name}" for col in accel_gyro_data.columns]
    #print(auc_values)
    auc_df = pd.DataFrame(auc_values).T
    auc_df.columns = columns
    
    return auc_df



# features to analyse
#mean
def feature_mean(feature_name,accel_gyro_data):
    mean = accel_gyro_data.mean()
    mean_transposed_df = mean.to_frame().T
    mean_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return mean_transposed_df

#std
def feature_std(feature_name,accel_gyro_data):
    std = accel_gyro_data.std()
    std_transposed_df = std.to_frame().T
    std_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return std_transposed_df

#kurtosis
def feature_kurtosis(feature_name,accel_gyro_data):
    kurtosis = accel_gyro_data.kurtosis()
    kurtosis_transposed_df = kurtosis.to_frame().T
    kurtosis_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return kurtosis_transposed_df

#median
def feature_median(feature_name,accel_gyro_data):
    median = accel_gyro_data.median()
    median_transposed_df = median.to_frame().T
    median_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return median_transposed_df

#skewness
def feature_skewness(feature_name,accel_gyro_data):
    skewness = accel_gyro_data.skew()
    skewness_transposed_df = skewness.to_frame().T
    skewness_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return skewness_transposed_df

# vector distance mean
def feature_sum(feature_name,accel_gyro_data):
    sum_ = accel_gyro_data.sum()
    sum_transposed_df = sum_.to_frame().T
    sum_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return sum_transposed_df

def calculate_zero_crossing_rate(accel_data):
    accel_data = (accel_data - np.mean(accel_data)) / np.std(accel_data)
        # Calculate Zerocrossing rate.
    zero_crossings = np.nonzero(np.diff(np.signbit(accel_data)))[0]
    zcr = len(zero_crossings) / (2.0 * len(accel_data))
    return zcr

def feature_zero_cross(feature_name,accel_gyro_data):
    zero_crossing_rates = {}
    for column in accel_gyro_data.columns:
        zcr = calculate_zero_crossing_rate(accel_gyro_data[column])
        zero_crossing_rates[column] = zcr

    zcr_series = pd.Series(zero_crossing_rates)
    zcr_transposed_df = zcr_series.to_frame().T
    zcr_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return zcr_transposed_df


# vector distance mean
def diff_sum(feature_name,accel_gyro_data):
    sum_ = accel_gyro_data.diff().sum()
    sum_transposed_df = sum_.to_frame().T
    sum_transposed_df.columns = ["Accel_X_"+feature_name ,"Accel_Y_" +feature_name ,"Accel_Z_" +feature_name,"Gyro_X_" +feature_name, "Gyro_Y_" + feature_name ,"Gyro_Z_" +feature_name ,"Magno_X_" +feature_name, "Magno_Y_" +feature_name ,"Magno_Z_" +feature_name, "Accel_Vecdis_" +feature_name, "Gyro_Vecdis_" +feature_name ,"Magno_Vecdis_" +feature_name]
    return sum_transposed_df

   
# create table 
def get_concated_features_2(not_normalized_accel_gyro_data,accel_gyro_data,sampling_rate):
    
    ma_not_normalized = moving_average_dataframe(not_normalized_accel_gyro_data, 5)
    ma_normalized_data = scaler.fit_transform(ma_not_normalized)
    ma_normalized = pd.DataFrame(ma_normalized_data, columns=ma_not_normalized.columns)
    
    df9 = dominant_frequency(ma_normalized, sampling_rate, 'dominant_freq')
    df10 = maximum_amplitude(ma_normalized, sampling_rate, 'max_amplitude')
    #df11 = calculate_q_factor(not_normalized_accel_gyro_data, sampling_rate, 'q_factor')
    

    
    accel_gyro_data['Accel'] = np.sqrt(np.sum(accel_gyro_data.iloc[:, 0:3] ** 2, axis = 1))
    accel_gyro_data['Gyro'] = np.sqrt(np.sum(accel_gyro_data.iloc[:, 3:6] ** 2, axis = 1))
    accel_gyro_data['Magno'] = np.sqrt(np.sum(accel_gyro_data.iloc[:, 6:9] ** 2, axis = 1))
    
    not_normalized_accel_gyro_data['Accel'] = np.sqrt(np.sum(not_normalized_accel_gyro_data.iloc[:, 0:3] ** 2, axis = 1))
    not_normalized_accel_gyro_data['Gyro'] = np.sqrt(np.sum(not_normalized_accel_gyro_data.iloc[:, 3:6] ** 2, axis = 1))
    not_normalized_accel_gyro_data['Magno'] = np.sqrt(np.sum(not_normalized_accel_gyro_data.iloc[:, 6:9] ** 2, axis = 1))
    
    ma_not_normalized_2 = moving_average_dataframe(not_normalized_accel_gyro_data, 5)
    ma_normalized_data_2 = scaler.fit_transform(ma_not_normalized_2)
    ma_normalized_2 = pd.DataFrame(ma_normalized_data_2, columns=ma_not_normalized_2.columns)
    
    
    fft = calculate_fft(ma_normalized_2, sampling_rate)
    
    df12 = feature_mean("mean_fft",fft)
    df13 = feature_median("median_fft",fft)
    df14 = feature_std("std_fft",fft)
    df15 = feature_kurtosis("kurtosis_fft",fft)
    df16 =feature_skewness("skewness_fft",fft)
    df17 = feature_sum("sum_fft",fft)
    df18 = feature_zero_cross("zero_cross_rate_fft", fft)
    df19 = diff_sum("diff_sum_fft", fft)
    df20 = calculate_auc_feature('AUC_fft', fft)
    
    df1 = feature_mean("mean",accel_gyro_data)
    df2 = feature_median("median",accel_gyro_data)
    df3 = feature_std("std",accel_gyro_data)
    df4 = feature_kurtosis("kurtosis",accel_gyro_data)
    df5 =feature_skewness("skewness",accel_gyro_data)
    df6 = feature_sum("sum",accel_gyro_data)
    df7 = feature_zero_cross("zero_cross_rate", not_normalized_accel_gyro_data)
    df8 = diff_sum("diff_sum", accel_gyro_data)

    df = [df1, df2, df3, df4, df5, df6,df7, df8,df9, df10,df12,df13,df14,df15,df16,df17,df18,df19,df20]
    result_df = pd.concat(df, axis=1)

    return result_df
    

In [1]:
scaler = MinMaxScaler()
directory_path = 'C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\BlobsOFSegments'

# Create an empty list to store the DataFrames
dataframes = []

# Loop through the files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Read the CSV file into a DataFrame and append it to the list
        df = pd.read_csv(file_path,low_memory=False)
        print(filename)
        
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns='Unnamed: 0')
        

        df_not_normalized = df.iloc[:, 4:13]
        #df_normalized = pd.DataFrame(scaler.fit_transform(df.iloc[:, 4:13]), columns=df.columns)
        df_normalized = pd.DataFrame(scaler.fit_transform(df.iloc[:, 4:13]), columns=df.iloc[:, 4:13].columns)
        sampling_rate  = calculate_sampling_frequency(df)
        hyper_result = get_concated_features_2(df_not_normalized,df_normalized ,sampling_rate)

        resulting_df = hyper_result#pd.concat(hyper_result, axis=0)

        if 'NoHyp' in filename:
            resulting_df['type'] = 0
        else:
            resulting_df['type'] = 1

        # split file name and add columns
        file_components = filename.split('_')
        resulting_df['Participent'] = file_components[0]
        resulting_df['Watch ID'] = file_components[1]
        resulting_df['Activity'] = file_components[2]
        
        resulting_df_new_1 = add_gender_age(resulting_df,filename)
        resulting_df_new = add_specific_hyperactive_behavior(resulting_df_new_1,filename)
        behavior_string = ' '.join(file_components[3:])
        resulting_df_new['Behavior'] = behavior_string
        
        dataframes.append(resulting_df_new)
        
        

resulting_df_final = pd.concat(dataframes, axis=0)
resulting_df_final.head()
#print(dataframes[0])  
resulting_df_final.to_csv('C:\\Users\\fried\\Documents\\URI\\ELE_591\\syncedfiles\\analysis_table_blobs_of_segments_with_mov_average_freq.csv', index=False)

NameError: name 'MinMaxScaler' is not defined

In [8]:
def moving_average_dataframe(X, window_size):
    X_new = pd.DataFrame()  # Create an empty DataFrame to store the results
    for column in X.columns:
        X_new[column] = X[column].rolling(window=window_size).mean()
    # Drop rows with NaN values
    X_new = X_new.dropna()
    return X_new




ma_not_normalized = moving_average_dataframe(df_not_normalized,5)
ma_normalized_data = scaler.fit_transform(ma_not_normalized)
ma_normalized = pd.DataFrame(ma_normalized_data, columns=ma_not_normalized.columns)
ma_normalized.head(20)


,Accel_X,Accel_Y,Accel_Z,Gyro_X,Gyro_Y,Gyro_Z,Magno_X,Magno_Y,Magno_Z,Accel,Gyro,Magno
0,0.593413,0.550690,0.740733,0.656350,0.671607,0.715695,0.436727,0.302746,0.106967,0.505830,0.127891,0.334348
1,0.582635,0.566075,0.757582,0.690486,0.679179,0.691433,0.436251,0.284330,0.142153,0.522639,0.128037,0.190713
2,0.566267,0.598028,0.777602,0.706576,0.685467,0.661319,0.426736,0.260420,0.179451,0.530832,0.111665,0.000000
3,0.561876,0.660355,0.794450,0.743393,0.704588,0.611217,0.424833,0.235218,0.164673,0.501220,0.115441,0.047500
4,0.553892,0.764892,0.791278,0.781664,0.733397,0.544020,0.429115,0.202262,0.128079,0.389446,0.145058,0.209829
5,0.545908,0.745957,0.779187,0.813230,0.753032,0.470840,0.439106,0.176414,0.101337,0.391155,0.185615,0.366950
6,0.541317,0.879684,0.788107,0.938041,0.785627,0.389835,0.453378,0.141519,0.066854,0.299389,0.320052,0.588005
7,0.566467,0.949507,0.765709,1.000000,0.793391,0.325728,0.480495,0.097900,0.055595,0.179955,0.401276,0.801501
8,0.661876,0.667061,0.873538,0.993240,0.815464,0.293445,0.506660,0.056220,0.070373,0.703714,0.422030,0.915206
9,0.666866,0.801578,0.921506,0.953908,0.878537,0.312841,0.545195,0.023263,0.107671,0.732823,0.436930,1.000000
